In [1]:
api_key = 'RGAPI-61349e6d-a977-4ec6-a52a-05d6bb4ad1d6'

In [2]:
import requests
import os
from psycopg2 import pool
from dotenv import load_dotenv
from psycopg2.extras import Json, DictCursor
import time

In [9]:
r = requests.get("https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/clu/clu1?api_key=" + api_key)
print(r.json())

{'puuid': 'R8_dzuvxZm0VsqU-paXM3wijLavgPmiZDmKIHFm8HddnUGvsgHTlCQuWdlHNRJlBIaEFyZYD87Avug', 'gameName': 'clu', 'tagLine': 'clu1'}


In [3]:
# setsuko, soju, dishsoap
streamers = ['e62ZthNQBc1bjVfXrjDvNIQV-0Bd7GUeaRv_fiZqx5nc5OXaY6OAmM4wav043AJgxKQShtA1s9IIOg', 'hVi-4fc19ymaRf4HvUmHNkZCoALJPUXgGKcKGgBlLZTqYZRVtQwglGQCIBj0jou0wA35CcH8sdeEgw', 'NUnja2Bl0CPRiimqr1PesnOnDsxXa6eAg5xRhNVBdxgAzBopnpcWNTZZ9r9YjNhEtf5tx2DkvXdJ9Q']

In [11]:
# for streamer_puuid in streamers:
#   matches = requests.get("https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + streamer_puuid + "&api_key=" + api_key)


In [12]:


# Load .env file
load_dotenv()

# Get the connection string from the environment variable
connection_string = os.getenv('DATABASE_URL')

# Create a connection pool
connection_pool = pool.SimpleConnectionPool(
    1,  # Minimum number of connections in the pool
    10,  # Maximum number of connections in the pool
    connection_string
)

# Check if the pool was created successfully
if connection_pool:
    print("Connection pool created successfully")

# Get a connection from the pool
conn = connection_pool.getconn()

# Create a cursor object
cur = conn.cursor()

# Execute SQL commands to retrieve the current time and version from PostgreSQL
cur.execute('SELECT NOW();')
time = cur.fetchone()[0]

cur.execute('SELECT version();')
version = cur.fetchone()[0]

# Close the cursor and return the connection to the pool
cur.close()
connection_pool.putconn(conn)

# Close all connections in the pool
connection_pool.closeall()

# Print the results
print('Current time:', time)
print('PostgreSQL version:', version)

Connection pool created successfully
Current time: 2024-11-12 00:23:18.013938+00:00
PostgreSQL version: PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit


In [5]:
for current_puuid in streamers:
  # Connect to the DB
  # Load .env file
  load_dotenv()

  # Get the connection string from the environment variable
  connection_string = os.getenv('DATABASE_URL')

  # Create a connection pool
  connection_pool = pool.SimpleConnectionPool(
      1,  # Minimum number of connections in the pool
      10,  # Maximum number of connections in the pool
      connection_string
  )

  # Check if the pool was created successfully
  if connection_pool:
      print("Connection pool created successfully")

  # Get a connection from the pool
  conn = connection_pool.getconn()
  conn.autocommit = True

  # Create a cursor object
  cur = conn.cursor(cursor_factory=DictCursor)


  # get recent match data
  matches = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/' + current_puuid + '/ids?start=0&count=20&api_key=' + api_key).json()

  # add each match to the db
  for match in matches:
    # first check not in the db
    
    cur.execute('SELECT 1 FROM matches WHERE match_id=%s', (match, ))
    if cur.fetchall():
      continue
    


    match_dict = {}
    match_data = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key)

    if match_data.ok:
      match_data = match_data.json()
    else:
      print("sleeping 120s")
      time.sleep(120)
      match_data = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key).json()


    #get metadata
    patch = match_data['info']['game_version'][-6:-1]
    game_datetime = match_data['info']['game_datetime']

    game_information = match_data['info']['participants']
    players = []
    # print(game_information[0])
    for board in game_information:
      curr_dict = {}
      # augments got removed saj
      # curr_dict['augments'] = board['augments']
      curr_dict['level'] = board['level']
      curr_dict['placement'] = board['placement']
      curr_dict['traits'] = board['traits']
      curr_dict['units'] = board['units']
      curr_dict['puuid'] = board['puuid']
      curr_dict['gold_left'] = board['gold_left']

      r = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/' + curr_dict['puuid'] + '?api_key=' + api_key)
      if r.ok:
        r = r.json()
      else:
        print("sleeping 120s")
        time.sleep(120)
        r = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/' + curr_dict['puuid'] + '?api_key=' + api_key).json()
      curr_dict['username_tagline']= r['gameName'] + ' #' + r["tagLine"]

      match_dict[r['gameName'] + ' #' + r["tagLine"]] = curr_dict.copy()
      players.append(r['gameName'] + ' #' + r["tagLine"])

    # add it into the db
  
    cur.execute('INSERT into matches (match_id, patch, game_datetime, match_data, players) values (%s, %s, %s, %s, %s)', (match, patch, game_datetime, Json(match_dict), players))
    print('inserted', match)



  # Commit changes to the db

  cur.close()
  connection_pool.putconn(conn)

  # Close all connections in the pool
  connection_pool.closeall()


Connection pool created successfully
inserted NA1_5170761962
inserted NA1_5170709108
inserted NA1_5170662524
inserted NA1_5170623468
inserted NA1_5170534476
inserted NA1_5170492053
inserted NA1_5170453249
inserted NA1_5170037493
inserted NA1_5170023972
inserted NA1_5169992232
sleeping 120s
inserted NA1_5169968181
inserted NA1_5169934585
inserted NA1_5169907124
inserted NA1_5169870177
inserted NA1_5169823671
inserted NA1_5169781121
inserted NA1_5169720573
inserted NA1_5169673483
inserted NA1_5169634333
inserted NA1_5169598478
Connection pool created successfully
inserted NA1_5170580491
sleeping 120s
inserted NA1_5170552980
inserted NA1_5170515264
inserted NA1_5170477331
inserted NA1_5170444680
inserted NA1_5170401862
inserted NA1_5170367955
inserted NA1_5170334704
inserted NA1_5170306704
inserted NA1_5170275399
inserted NA1_5170252026
inserted NA1_5170222485
sleeping 120s
inserted NA1_5170204098
inserted NA1_5170176709
inserted NA1_5170158405
inserted NA1_5170138203
inserted NA1_5170123

In [37]:
def getStreamerData(username, page):
    page_size = 5
    load_dotenv()

    # Get the connection string from the environment variable
    connection_string = os.getenv('DATABASE_URL')

    # Create a connection pool
    connection_pool = pool.SimpleConnectionPool(
        1,  # Minimum number of connections in the pool
        10,  # Maximum number of connections in the pool
        connection_string
    )

    # Check if the pool was created successfully
    if connection_pool:
        print("Connection pool created successfully")

    # Get a connection from the pool
    conn = connection_pool.getconn()
    conn.autocommit = True

    # Create a cursor object
    cur = conn.cursor(cursor_factory=DictCursor)

    cur.execute("SELECT match_data FROM matches WHERE '%s' = ANY(players) ORDER BY game_datetime DESC" % (username))
    res = cur.fetchall()
    cur.close()
    connection_pool.putconn(conn)

    # Close all connections in the pool
    connection_pool.closeall()

    from functools import reduce
    return reduce(lambda x,y :x+y , res[page*5:(page+1)*5])
getStreamerData("Knav #NA1", 0)

Connection pool created successfully


IndexError: list index out of range

In [19]:
# units is a list of dicts
def findComp(units, synergies, level_carries, reroll_carries, synergy_dict, unit_dict):
    ## if there is a silver/gold synergy, add it to name, otherwise flex
    ## if there is 4 cost carry with 3 items, add it to comp name
    ## if lots of 2 star 5 cost add "+ legendaries"
    ## returns a list of comps for easy filtering
    ## if 3 star unit with 3 items, add reroll
    comp = []

    for synergy in synergies:
        if synergy["tier_current"] >= 2:
            # comp.append(synergy_dict[synergy["name"] + synergy["tier_current"]])
            comp.append(synergy["name"] + str(synergy["tier_current"]))
    reroll = False
    for unit in units:
        if unit["character_id"] in reroll_carries and unit["tier"] == 3 and len(unit["itemNames"]) == 3:
            reroll = True
    for unit in units:
        if reroll and unit["tier"] >= 3 and len(unit["itemNames"]) == 3:
            # comp.append(unit_dict[unit["character_id"]])
            comp.append(unit["character_id"])
        elif not reroll and unit["character_id"] in level_carries and len(unit["itemNames"]) == 3:
            # comp.append(unit_dict[unit["character_id"]])
            comp.append(unit["character_id"])

    if reroll:
        comp.append("Reroll")
    return comp


In [20]:
units = [
      {
        "name": "",
        "tier": 3,
        "rarity": 0,
        "itemNames": [
          "TFT13_Item_PitFighterEmblemItem"
        ],
        "character_id": "TFT13_Darius"
      },
      {
        "name": "",
        "tier": 3,
        "rarity": 0,
        "itemNames": [
          "TFT_Item_HextechGunblade",
          "TFT_Item_RunaansHurricane",
          "TFT_Item_UnstableConcoction"
        ],
        "character_id": "TFT13_Draven"
      },
      {
        "name": "",
        "tier": 3,
        "rarity": 0,
        "itemNames": [],
        "character_id": "TFT13_Blue"
      },
      {
        "name": "",
        "tier": 4,
        "rarity": 0,
        "itemNames": [
          "TFT_Item_TitansResolve",
          "TFT_Item_SteraksGage",
          "TFT_Item_Bloodthirster"
        ],
        "character_id": "TFT13_Red"
      },
      {
        "name": "",
        "tier": 2,
        "rarity": 1,
        "itemNames": [
          "TFT_Item_Redemption",
          "TFT4_Item_OrnnRanduinsSanctum"
        ],
        "character_id": "TFT13_Prime"
      },
      {
        "name": "",
        "tier": 2,
        "rarity": 2,
        "itemNames": [
          "TFT_Item_NeedlesslyLargeRod",
          "TFT7_Item_ShimmerscaleHeartOfGold"
        ],
        "character_id": "TFT13_Gangplank"
      },
      {
        "name": "",
        "tier": 2,
        "rarity": 4,
        "itemNames": [
          "TFT_Item_ThiefsGloves",
          "TFT_Item_EmptyBag",
          "TFT_Item_EmptyBag"
        ],
        "character_id": "TFT13_Vi"
      },
      {
        "name": "",
        "tier": 2,
        "rarity": 6,
        "itemNames": [
          "TFT_Item_Bloodthirster",
          "TFT_Item_TitansResolve",
          "TFT13_Item_FamilyEmblemItem"
        ],
        "character_id": "TFT13_Lieutenant"
      },
      {
        "name": "",
        "tier": 2,
        "rarity": 6,
        "itemNames": [],
        "character_id": "TFT13_Lieutenant"
      }
    ]

synergies = [
      {
        "name": "TFT13_Ambusher",
        "style": 0,
        "num_units": 1,
        "tier_total": 4,
        "tier_current": 0
      },
      {
        "name": "TFT13_Crime",
        "style": 0,
        "num_units": 1,
        "tier_total": 5,
        "tier_current": 0
      },
      {
        "name": "TFT13_Family",
        "style": 4,
        "num_units": 4,
        "tier_total": 3,
        "tier_current": 2
      },
      {
        "name": "TFT13_FormSwapper",
        "style": 0,
        "num_units": 1,
        "tier_total": 2,
        "tier_current": 0
      },
      {
        "name": "TFT13_HighRoller",
        "style": 3,
        "num_units": 1,
        "tier_total": 1,
        "tier_current": 1
      },
      {
        "name": "TFT13_Pugilist",
        "style": 4,
        "num_units": 6,
        "tier_total": 4,
        "tier_current": 3
      },
      {
        "name": "TFT13_Scrap",
        "style": 1,
        "num_units": 2,
        "tier_total": 4,
        "tier_current": 1
      },
      {
        "name": "TFT13_Squad",
        "style": 0,
        "num_units": 1,
        "tier_total": 5,
        "tier_current": 0
      },
      {
        "name": "TFT13_Warband",
        "style": 1,
        "num_units": 2,
        "tier_total": 4,
        "tier_current": 1
      },
      {
        "name": "TFT13_Watcher",
        "style": 1,
        "num_units": 2,
        "tier_total": 3,
        "tier_current": 1
      }
    ]

level_carries = set(["TFT13_Silco", "TFT13_Vi", "TFT13_Caitlyn", "TFT13_Ekko", "TFT13_Malzahar", "TFT13_Twitch", "TFT13_Cassiopeia", "TFT13_LeBlanc", "TFT13_Heimerdinger", "TFT13_Jayce", "TFT13_Lieutenant", "TFT13_Jinx", "TFT13_Nami", "TFT13_Corki", "TFT13_Ambessa", "TFT13_Mordekaiser", "TFT13_Zoe"])
reroll_carries = set(["TFT13_Camille", "TFT13_RenataGlasc", "TFT13_Akali", "TFT13_Ezreal", "TFT13_Draven", "TFT13_Cassiopeia", "TFT13_Ziggs", "TFT13_Gremlin", "TFT13_Shooter", "TFT13_Blue", "TFT13_Zeri", "TFT13_Red", "TFT13_Nami", "TFT13_Vex", "TFT13_Nocturne", "TFT13_Darius", "TFT13_Irelia", "TFT13_FlyGuy", "TFT13_NunuWillump", "TFT13_Prime", "TFT13_Beardy", "TFT13_Chainsaw", "TFT13_Vladimir", "TFT13_Rell", "TFT13_Sett", "TFT13_Amumu", "TFT13_Blitzcrank", "TFT13_Singed", "TFT13_Zyra", "TFT13_Gangplank", "TFT13_Leona", "TFT13_KogMaw", "TFT13_TwistedFate", "TFT13_Lux", "TFT13_Morgana", "TFT13_Tristana", "TFT13_Swain", "TFT13_Urgot", ""])
synergy_dict = {}
unit_dict = {"TFT13_Silco": "Silco", "TFT13_Caitlyn": "Caitlyn", "TFT13_Ekko": "Ekko", "TFT13_Malzahar": "Malzahar",
             "TFT13_Vi": "Vi", 
             }

findComp(units, synergies, level_carries, reroll_carries, synergy_dict, unit_dict)

['TFT13_Family2', 'TFT13_Pugilist3', 'TFT13_Draven', 'TFT13_Red', 'Reroll']

In [10]:
def updateData(username):
    # Connect to the DB
    # Load .env file
    load_dotenv()

    # Get the connection string from the environment variable
    connection_string = os.getenv('DATABASE_URL')

    # Create a connection pool
    connection_pool = pool.SimpleConnectionPool(
        1,  # Minimum number of connections in the pool
        10,  # Maximum number of connections in the pool
        connection_string
    )

    # Check if the pool was created successfully
    if connection_pool:
        print("Connection pool created successfully")

    # Get a connection from the pool
    conn = connection_pool.getconn()
    conn.autocommit = True

    # Create a cursor object
    cur = conn.cursor(cursor_factory=DictCursor)

    # get recent match data
    split_username = username.split(" ")
    print(split_username)
    current_puuid = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/%s/%s?api_key=%s' % (split_username[0], split_username[1][1:], api_key)).json()['puuid']
    matches = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/' + current_puuid + '/ids?start=0&count=20&api_key=' + api_key).json()

    # add each match to the db
    for match in matches:
        # first check not in the db

        cur.execute('SELECT 1 FROM matches WHERE match_id=%s', (match, ))
        if cur.fetchall():
            continue

        match_dict = {}
        match_data = requests.get(
            'https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key)

        if match_data.ok:
            match_data = match_data.json()
        else:
            print("sleeping 120s")
            time.sleep(120)
            match_data = requests.get(
                'https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key).json()

        # get metadata
        patch = match_data['info']['game_version'][-6:-1]
        game_datetime = match_data['info']['game_datetime']

        game_information = match_data['info']['participants']
        players = []
        # print(game_information[0])
        for board in game_information:
            curr_dict = {}
            curr_dict['augments'] = board['augments']
            curr_dict['level'] = board['level']
            curr_dict['placement'] = board['placement']
            curr_dict['traits'] = board['traits']
            curr_dict['units'] = board['units']
            curr_dict['puuid'] = board['puuid']
            curr_dict['gold_left'] = board['gold_left']

            r = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/' +
                             curr_dict['puuid'] + '?api_key=' + api_key)
            if r.ok:
                r = r.json()
            else:
                print("sleeping 120s")
                time.sleep(120)
                r = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/' +
                                 curr_dict['puuid'] + '?api_key=' + api_key).json()
            curr_dict['username_tagline'] = r['gameName'] + ' #' + r["tagLine"]

            match_dict[r['gameName'] + ' #' + r["tagLine"]] = curr_dict.copy()
            players.append(r['gameName'] + ' #' + r["tagLine"])

        # add it into the db

        cur.execute('INSERT into matches (match_id, patch, game_datetime, match_data, players) values (%s, %s, %s, %s, %s)',
                    (match, patch, game_datetime, Json(match_dict), players))
        print('inserted', match)

    # Commit changes to the db

    cur.close()
    connection_pool.putconn(conn)

    # Close all connections in the pool
    connection_pool.closeall()


updateData("clu #clu1")

Connection pool created successfully
['clu', '#clu1']
inserted NA1_5128344966
inserted NA1_5126157012
inserted NA1_5124132700
inserted NA1_5124114670
inserted NA1_5124093511
inserted NA1_5122870794
inserted NA1_5122842071
inserted NA1_5122805672
inserted NA1_5122782530
inserted NA1_5122761335
sleeping 120s
inserted NA1_5122735271
